# 🎨 LangGraph Poster Generator - Google Colab

This notebook runs the full multi-agent poster generation pipeline on Google Colab.

**Pipeline Stages:**
1. **Planning Agent** - Analyzes inputs and creates design plan
2. **Text Generation** - Generates poster text with validation (max 3 attempts)
3. **Image Generation** - Creates poster image with validation (max 3 attempts)
4. **Text Addition** - Adds text to final image

**Requirements:**
- GPU Runtime (T4 recommended)
- OpenRouter API key
- HuggingFace token

---

## 📦 Step 1: Setup & Installation

Clone the repository and install dependencies.

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\nCUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Clone the repository (replace with your actual GitHub repo URL)
import os

REPO_URL = "https://github.com/YOUR_USERNAME/ARIN-5201-Project.git"  # UPDATE THIS!

# Remove existing directory if present
if os.path.exists("ARIN-5201-Project"):
    !rm -rf ARIN-5201-Project

# Clone repository
!git clone {REPO_URL}

# Change to project directory
%cd ARIN-5201-Project

In [ ]:
# Install all required dependencies
!pip install -q -r requirements.txt

print("✅ All dependencies installed!")

## 🔑 Step 2: Configure API Keys

Enter your API keys below:
- **OpenRouter API Key**: Get from [openrouter.ai](https://openrouter.ai/)
- **HuggingFace Token**: Get from [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)

In [ ]:
from getpass import getpass
import os

# Input API keys securely
OPENROUTER_API_KEY = getpass("Enter your OpenRouter API Key: ")
HUGGINGFACE_TOKEN = getpass("Enter your HuggingFace Token: ")

# Create .env file
with open(".env", "w") as f:
    f.write(f"OPENROUTER_API_KEY={OPENROUTER_API_KEY}\n")
    f.write(f"HUGGINGFACE_TOKEN={HUGGINGFACE_TOKEN}\n")

print("✅ API keys configured!")

## 📤 Step 3: Upload Inputs

Upload your logo/mascot image and enter keywords for the poster.

In [ ]:
from google.colab import files
from PIL import Image
import io

print("📸 Upload your logo/mascot image (PNG format):")
uploaded = files.upload()

# Get the first uploaded file
if uploaded:
    filename = list(uploaded.keys())[0]
    
    # Load and verify it's a valid image
    img = Image.open(io.BytesIO(uploaded[filename]))
    
    # Convert to PNG if needed and save as input.png
    if img.mode in ('RGBA', 'LA', 'P'):
        img.save("input.png", "PNG")
    else:
        # Convert to RGB first, then save
        img_rgb = img.convert("RGB")
        img_rgb.save("input.png", "PNG")
    
    print(f"✅ Image saved as input.png")
    print(f"   Size: {img.size}")
    
    # Display the uploaded image
    display(img)
else:
    print("❌ No image uploaded!")

In [ ]:
# Enter poster keywords/description
print("📝 Enter your poster keywords or description (2-5 keywords or short description):")
print("   Example: 'Summer Music Festival 2024, Outdoor Concert, Live Bands'")
print()

text_input = input("Keywords: ")

# Save as input.txt
with open("input.txt", "w") as f:
    f.write(text_input)

print(f"\n✅ Text saved as input.txt")
print(f"   Content: {text_input}")

## 🚀 Step 4: Run Pipeline

This will run the full multi-agent workflow. It may take 5-15 minutes depending on GPU availability.

In [ ]:
# Run the main pipeline
print("🎨 Starting poster generation pipeline...\n")
print("=" * 60)

!python main.py

print("\n" + "=" * 60)
print("✅ Pipeline completed!")

## 🎉 Step 5: View Results

Display the generated poster and intermediate outputs.

In [ ]:
from PIL import Image
from IPython.display import display, Markdown
import os

print("📊 FINAL OUTPUTS\n")
print("=" * 60)

# Display final poster
if os.path.exists("outputs/poster.png"):
    print("\n🖼️  Final Poster:")
    poster = Image.open("outputs/poster.png")
    display(poster)
else:
    print("❌ Final poster not found!")

# Display generated text
if os.path.exists("outputs/text.txt"):
    print("\n📝 Generated Text:")
    print("-" * 60)
    with open("outputs/text.txt", "r") as f:
        text_content = f.read()
    print(text_content)
    print("-" * 60)
else:
    print("❌ Generated text not found!")

print("\n" + "=" * 60)

In [ ]:
from PIL import Image
from IPython.display import display
import os

print("🔍 INTERMEDIATE OUTPUTS\n")
print("=" * 60)

# Display planning output
if os.path.exists("intermediate_outputs/planning.txt"):
    print("\n📋 Planning Agent Output:")
    print("-" * 60)
    with open("intermediate_outputs/planning.txt", "r") as f:
        planning = f.read()
    print(planning[:500] + "..." if len(planning) > 500 else planning)
    print("-" * 60)

# Display text generation attempts
print("\n📝 Text Generation Attempts:")
for i in range(1, 4):
    attempt_file = f"intermediate_outputs/text_attempt{i}.txt"
    if os.path.exists(attempt_file):
        print(f"\n  Attempt {i}:")
        with open(attempt_file, "r") as f:
            text = f.read()
        print(f"  {text[:200]}..." if len(text) > 200 else f"  {text}")

# Display image generation attempts
print("\n🖼️  Image Generation Attempts:")
for i in range(1, 4):
    attempt_file = f"intermediate_outputs/attempt{i}.png"
    if os.path.exists(attempt_file):
        print(f"\n  Attempt {i}:")
        img = Image.open(attempt_file)
        # Resize for display
        img.thumbnail((400, 400))
        display(img)

# Display poster with text
if os.path.exists("intermediate_outputs/poster_with_text.png"):
    print("\n🎨 Poster with Text (before final output):")
    img = Image.open("intermediate_outputs/poster_with_text.png")
    display(img)

print("\n" + "=" * 60)

## 💾 Step 6: Download Results

Download the final poster and all outputs.

In [ ]:
from google.colab import files
import os

print("📥 Downloading outputs...\n")

# Download final poster
if os.path.exists("outputs/poster.png"):
    files.download("outputs/poster.png")
    print("✅ Downloaded: poster.png")

# Download final text
if os.path.exists("outputs/text.txt"):
    files.download("outputs/text.txt")
    print("✅ Downloaded: text.txt")

print("\n🎉 All downloads complete!")

In [ ]:
# Optional: Create a zip file with all outputs
import shutil
from google.colab import files

print("📦 Creating zip file with all outputs...\n")

# Create zip file
shutil.make_archive("poster_outputs", "zip", "outputs")
shutil.make_archive("intermediate_outputs_archive", "zip", "intermediate_outputs")

# Download zip files
files.download("poster_outputs.zip")
files.download("intermediate_outputs_archive.zip")

print("✅ Downloaded: poster_outputs.zip")
print("✅ Downloaded: intermediate_outputs_archive.zip")
print("\n🎉 All files packaged and downloaded!")

## 🧹 Step 7: Cleanup (Optional)

Clear outputs to prepare for a new run.

In [ ]:
import shutil
import os

print("🧹 Cleaning up outputs...\n")

# Clear output directories
if os.path.exists("outputs"):
    shutil.rmtree("outputs")
    os.makedirs("outputs")
    print("✅ Cleared outputs/")

if os.path.exists("intermediate_outputs"):
    shutil.rmtree("intermediate_outputs")
    os.makedirs("intermediate_outputs")
    print("✅ Cleared intermediate_outputs/")

# Remove input files
if os.path.exists("input.png"):
    os.remove("input.png")
    print("✅ Removed input.png")

if os.path.exists("input.txt"):
    os.remove("input.txt")
    print("✅ Removed input.txt")

print("\n🎉 Cleanup complete! Ready for a new run.")

---

## 📚 Additional Information

**Troubleshooting:**
- If you get CUDA out of memory errors, restart runtime and try again
- Model downloads on first run (~2-5 GB), subsequent runs will use cached model
- Ensure you're using a GPU runtime (Runtime → Change runtime type → T4 GPU)

**Pipeline Details:**
- Text generation has max 3 retry attempts with validation
- Image generation has max 3 retry attempts with validation
- If max retries reached, system continues with best attempt
- All intermediate outputs saved for debugging

**Project Repository:** [GitHub Link](https://github.com/YOUR_USERNAME/ARIN-5201-Project)

**Course:** ARIN-5201 Course Project - Creative Poster Generator

---